# WaveNet - Fit a Sample

In [1]:
import sys
#sys.path.append('../../src/')
sys.path.append('../../network/')

In [2]:
import os
import torch
from types import SimpleNamespace
torch.cuda.empty_cache()

In [3]:
from models.wavenet_no_padding.model import WaveNet
from models.wavenet_no_padding.utils.data import DataLoader

In [4]:
params = SimpleNamespace(
    layer_size=10,
    stack_size=5,
    in_channels=256,
    res_channels=512,
    lr=2e-3,
    sample_size=15_000,
    sample_rate=22_050,
    epochs=10_000,
    model_dir='../../network/weights/wavenet/'
)

In [5]:
class Trainer:
    def __init__(self,
                 layer_size: int = 10,
                 stack_size: int = 5,
                 in_channels: int = 256,
                 res_channels: int = 512,
                 lr: float = 2e-3,
                 sample_size: int = 100_000,
                 sample_rate: int =22_050,
                 epochs: int = 10_000,
                 data_dir: str = '.',
                 model_dir: str = './',
                 model_name: str = None):
        """
        """
        self.epochs = epochs
        self.model_dir = model_dir
        self.model_name = model_name
        
        self.wavenet = WaveNet(layer_size, stack_size, in_channels, res_channels, lr=lr)

        self.data_loader = DataLoader(data_dir, self.wavenet.receptive_fields,
                                      sample_size, sample_rate, in_channels)

    def infinite_batch(self):
        while True:
            for dataset in self.data_loader:
                for inputs, targets in dataset:
                    yield inputs, targets

    def run(self):
        total_steps = 0

        for inputs, targets in self.infinite_batch():
            loss = self.wavenet.train(inputs, targets)

            total_steps += 1

            print('[{0}/{1}] loss: {2}'.format(total_steps, self.epochs, loss))

            if total_steps > self.epochs:
                break

            if total_steps % 200 == 0:
                self.wavenet.save(self.model_dir, self.model_name)

In [6]:
#params.data_dir = '../../data/processed/tapping/tapping-glass/partial/'
#params.model_name = 'wavenet-tapping-glass-tiny-jar-2'
params.data_dir = '../../data/processed/tapping/tapping-glass/partial/'
params.model_name = 'wavenet-tapping-glass-no-padding'

In [7]:
trainer = Trainer(**params.__dict__)
#trainer.wavenet.load(params.model_dir, params.model_name)
trainer.run()

RuntimeError: CUDA error: out of memory